# LightGBM with Optuna tuning
* doc: 
https://github.com/optuna/optuna

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import random
import optuna
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-jul-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")

In [ ]:
train.head()

In [ ]:
print(train.columns.to_list())

In [ ]:
target_name=['target_carbon_monoxide','target_benzene','target_nitrogen_oxides']
train_name=['deg_C','relative_humidity','absolute_humidity','sensor_1','sensor_2',
            'sensor_3','sensor_4','sensor_5',]

In [ ]:
target = train[target_name]
target0 = train['target_carbon_monoxide']
data = train[train_name]

In [ ]:
columns=data.columns.to_list()
print(columns)

In [ ]:
def objective(trial,data=data,target=target0):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param =   {

        'max_depth':  trial.suggest_int('max_depth', 8, 10),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.3, 0.4),
        'num_leaves': trial.suggest_int('num_leaves', 100, 256),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 0.01, 1.0),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 7.0, 8.0),
        "bagging_freq":  trial.suggest_int('num_leaves', 2, 256),
        "bagging_fraction": trial.suggest_uniform('bagging_fraction', 0.9, 1.0),
        "feature_fraction": trial.suggest_uniform('feature_fraction', 0.4, 0.5),
        "verbosity":  trial.suggest_int('verbosity', 3, 6),
        
        'objective': trial.suggest_categorical('objective',['regression']),         
        "boosting": trial.suggest_categorical('boosting',["gbdt"]),
        'tree_method': trial.suggest_categorical('tree_method',['gpu_hist']),  #'gpu_hist','hist'    
    }
    model = lgb.LGBMRegressor(**param)      
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    preds = model.predict(test_x)
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=8)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
study.trials_dataframe()

In [ ]:
# shows the scores from all trials
optuna.visualization.plot_optimization_history(study)

In [ ]:
# interactively visualizes the hyperparameters and scores
optuna.visualization.plot_parallel_coordinate(study)

In [ ]:
# shows the evolution of the search
optuna.visualization.plot_slice(study)

In [ ]:
# parameter interactions on an interactive chart.
optuna.visualization.plot_contour(study, params=['bagging_fraction','num_leaves'])

In [ ]:
# Visualize parameter importances.
optuna.visualization.plot_param_importances(study)

In [ ]:
# Visualize empirical distribution function
optuna.visualization.plot_edf(study)

In [ ]:
Best_trial=study.best_trial.params
print(Best_trial)

In [ ]:
sample = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")
print(sample.shape)
sample

In [ ]:
preds = np.zeros((sample.shape[0],sample.shape[1]-1))
kf = KFold(n_splits=5,random_state=48,shuffle=True)
for i in range(3):
    for trn_idx, test_idx in kf.split(train[columns],target[target_name[i]]):
        X_tr,X_val=train[columns].iloc[trn_idx],train[columns].iloc[test_idx]
        y_tr,y_val=target.iloc[trn_idx,i],target.iloc[test_idx,i]     ######
        model = lgb.LGBMRegressor(**Best_trial)
        model.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=100,verbose=False)
        preds[:,i]+=model.predict(test[columns])/kf.n_splits    ######
        rmse=mean_squared_error(y_val, model.predict(X_val),squared=False)
        print(rmse)

In [ ]:
subm = sample
subm[target_name] = pd.DataFrame(preds)
subm.to_csv('submission.csv',index=False)
subm